In [1]:
#https://carnegieendowment.org/publications#analysis

In [2]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")
today_date = date.today().strftime("%Y%m%d")
# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'carnegieendowment_org_publications_analysis.ipynbations_analysis'  #carnegieendowment.org/publications#analysis
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
publish_source = 'carnegieendowment.org'
country = 'United Kingdom'
language = 'English'
images_path = []
source_contents_list=['<p class="zn-body__paragraph speakable".*?>(.*?)<p class="zn-body__paragraph',
                      '<div class="entry-content content">(.*?)<hr',
                     '<div class="article-body">(.*?)<div class="section">',
                     '<div class="entry-content pf-content">(.*?)\.</li>',
                     'div class="body larger">(.*?)<div class=\'info-box bio\'>',
                     '<span class="dropcap dropcap2">(.*?)<p class="postBtm"',
                      '<div class="detail">(.*?)<div class="shareArticle">',
                      '"articleBody":"(.*?)"',
                     '<div class="article-body">(.*?)<div class="meta-heading-2">',                    
                     '<div class="styles-text">(.*?)</article>',
                      '<section class="c-content">(.*?)<section class="c-tags">',
                     '"description"\s*:\s*"(.*?)"']
# archive page
rss_page = c.download_page('https://carnegieendowment.org/publications#analysis')

for i in rss_page.split('<h4><a href=')[1:2]:
    
    # source_link
    source_link = c.scrap('"(.*?)"', i)
    if "https" not in source_link:
        source_link='https://carnegieendowment.org'+source_link
    if 'pdf' in source_link:  #pdf page
        continue
    if 'toward-just-us-nuclear' in source_link:  #it required login credentials
        continue

    # handle duplicates


    logger.info(f'Fetching {source_link}\n')
  
    page = c.download_page(source_link)
    
    print(source_link)
#     break
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue
        
    time.sleep(random.randint(1,3))
    
    source_headline = c.scrap('<meta property="og:title".*?="(.*?)"', page)
    if '<meta property="og:title' not in page:  # to check the condition and give the title
        source_headline = c.scrap('<title.*?>(.*?)</title>', page)
    source_headline =re.sub('&.*?;','',source_headline,re.S)
#     source_headline =re.sub('[Opinion]','',source_headline,)
    # skip if headline not found
#     print(source_headline)
#     continue
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

    # Date and time
    pub_date, publish_time = '', ''

    try:
        date_time_str = ''
        date_time_str = c.scrap('datePublished":\s*"(.*?)"',page)
        if not date_time_str:
            date_time_str = today_date
        date_time_str=re.sub('[^\w+]','',date_time_str,re.S)
        date_time_obj = datetime.strptime(date_time_str, '%Y%m%dT%H%M%S+0000')#20210219T134316
        ist_date_time = date_time_obj + timedelta(hours = 5,minutes = 30)  # utc time to ist
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]


    except:
        pub_date = today_date
        publish_time = current_time

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
#     if pub_date != created_on:
#         break    

    logger.info(f'Fetching {source_link}\n')
    
#     page = c.download_page(source_link)
    if page.startswith('Unable to fetch'):
        logger.info(page) # writes error message with error code
        unable_to_fetch_article_url += 1
        continue    
    source_content=''        
    for regex in source_contents_list:  # forming regex operation in the data using loop
        source_content=c.scrap(regex,page) #c.scrap(re.search(regex,page))
        # if sourec content is true, as per our condition,bec upar ka regex mil gya so it wil perform the operation
        if source_content:  # if regex found perform below operation,else search another regex
            source_content = re.sub('<li>','.',source_content,re.S) #&rsquo;
            source_content = re.sub('&rsquo','',source_content,re.S) #
            source_content = re.sub('&mdash',' ',source_content,re.S)
            source_content = re.sub(r'&rdquo;',' ',source_content,re.S)
            source_content = re.sub(r'&ndash;',' ',source_content,re.S)
            source_content = re.sub(r'&ldquo;',' ',source_content,re.S)
            source_content = re.sub(r'&rdquo;',' ',source_content,re.S)
            #&mdash;
            source_content = c.strip_html(source_content)
            print(source_content)
            break
#             print(source_content)
#             continue

    # skip if content not found
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')
        skipped_due_to_content += 1
        continue

    # journalist
    journalist = ''
    if not journalist: journalist = 'NA'

    # current date and time 
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
    
    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content
                } 

#     cl_data.insert_one(clientdata)  # get object id and insert data
    no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to article fetch url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://carnegieeurope.eu/2021/03/17/new-framework-for-dealing-with-venezuela-from-democracy-to-conflict-resolution-pub-84057
European Democracy Hub

Venezuela is mired in a prolonged, multifaceted crisis, to which no solutions are in sight. In the wake of the country;s December 2020 parliamentary election, the EU needs to rethink some of the basic premises of its policy toward Venezuela. Instead of quarreling about which domestic actors and political institutions should be recognized as democratic, the EU should approach the country through a lens of conflict resolution. While a democracy-based framework divides the EU and a broad range of other external actors, a framework focused on conflict resolution may increase the chances of a more coordinated international response. That approach may be more likely to lead ;eventually and indirectly ;to some kind of inclusive political settlement in Venezuela.

A Fresh Approach to a Protracted Stalemate

On December 6, 2020, Venezuela held a p

In [ ]:
#&lsquo;